In [1]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
print(f"Tensorflow {tf.__version__}")

2025-10-20 13:43:09.299823: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-20 13:43:09.301573: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-20 13:43:09.475215: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-20 13:43:10.927326: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To tur

Tensorflow 2.20.0


In [2]:
# Function to load the CIFAR-10 dataset
import pickle
from keras.utils import to_categorical
def unpickle(file):
    """load the cifar-10 data"""

    with open(file, 'rb') as fo:
        data = pickle.load(fo, encoding='bytes')
    return data


def load_cifar_10_data(data_dir, negatives=False):
    """
    Return train_data, train_filenames, train_labels, test_data, test_filenames, test_labels
    """

    # get the meta_data_dict
    # num_cases_per_batch: 1000
    # label_names: ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
    # num_vis: :3072

    meta_data_dict = unpickle(data_dir + "/batches.meta")
    cifar_label_names = meta_data_dict[b'label_names']
    cifar_label_names = np.array(cifar_label_names)

    # training data
    cifar_train_data = None
    cifar_train_filenames = []
    cifar_train_labels = []

    for i in range(1, 6):
        cifar_train_data_dict = unpickle(data_dir + "/data_batch_{}".format(i))
        if i == 1:
            cifar_train_data = cifar_train_data_dict[b'data']
        else:
            cifar_train_data = np.vstack((cifar_train_data, cifar_train_data_dict[b'data']))
        cifar_train_filenames += cifar_train_data_dict[b'filenames']
        cifar_train_labels += cifar_train_data_dict[b'labels']

    cifar_train_data = cifar_train_data.reshape((len(cifar_train_data), 3, 32, 32))
    if negatives:
        cifar_train_data = cifar_train_data.transpose(0, 2, 3, 1).astype(np.float32)
    else:
        cifar_train_data = np.rollaxis(cifar_train_data, 1, 4)
    cifar_train_filenames = np.array(cifar_train_filenames)
    cifar_train_labels = np.array(cifar_train_labels)

    cifar_test_data_dict = unpickle(data_dir + "/test_batch")
    cifar_test_data = cifar_test_data_dict[b'data']
    cifar_test_filenames = cifar_test_data_dict[b'filenames']
    cifar_test_labels = cifar_test_data_dict[b'labels']

    cifar_test_data = cifar_test_data.reshape((len(cifar_test_data), 3, 32, 32))
    if negatives:
        cifar_test_data = cifar_test_data.transpose(0, 2, 3, 1).astype(np.float32)
    else:
        cifar_test_data = np.rollaxis(cifar_test_data, 1, 4)
    cifar_test_filenames = np.array(cifar_test_filenames)
    cifar_test_labels = np.array(cifar_test_labels)

    return cifar_train_data, cifar_train_filenames, to_categorical(cifar_train_labels), \
        cifar_test_data, cifar_test_filenames, to_categorical(cifar_test_labels), cifar_label_names



In [3]:
import ANNarchy
from ANNarchy.extensions.ann_to_snn_conversion import ANNtoSNNConverter
ANNarchy.clear()
snn_converter = ANNtoSNNConverter(
    input_encoding='IB', 
    hidden_neuron='IaF',
    read_out='spike_count',
)


ANNarchy 4.8 (4.8.2.5) on linux (posix).


In [5]:
# Load the pretrained Keras model
model = tf.keras.models.load_model('pretrainedResnet.h5')
model.save('pretrainedResnet.keras')


/home/sist/projects/test_ann_to_snn/venv_annarchy/lib/python3.12/site-packages/keras/src/optimizers/base_optimizer.py:86: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


In [6]:
net = snn_converter.load_keras_model("pretrainedResnet.keras", show_info=True)


/home/sist/projects/test_ann_to_snn/venv_annarchy/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'Adam', because it has 70 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


* Input layer: input_1, (32, 32, 3)
* InputLayer skipped.
* Conv2D layer: conv2d, (32, 32, 16) 
* BatchNormalization skipped.
* Activation skipped.
* Conv2D layer: conv2d_1, (32, 32, 16) 
* BatchNormalization skipped.
* Activation skipped.
* Conv2D layer: conv2d_2, (32, 32, 16) 
* BatchNormalization skipped.
* Add skipped.
* Activation skipped.
* Conv2D layer: conv2d_3, (16, 16, 32) 
* BatchNormalization skipped.
* Activation skipped.
* Conv2D layer: conv2d_4, (16, 16, 32) 
* Conv2D layer: conv2d_5, (16, 16, 32) 
* BatchNormalization skipped.
* Add skipped.
* Activation skipped.
* Conv2D layer: conv2d_6, (8, 8, 64) 
* BatchNormalization skipped.
* Activation skipped.
* Conv2D layer: conv2d_7, (8, 8, 64) 
* Conv2D layer: conv2d_8, (8, 8, 64) 
* BatchNormalization skipped.
* Add skipped.
* Activation skipped.
* MaxPooling2D layer: average_pooling2d, (1, 1, 64) 
* Flatten skipped.
* Dense layer: dense, 10 
    weights: (10, 64)
    mean -0.16150876879692078, std 1.1581709384918213
    min

In [14]:
train_data, train_filenames, train_labels, test_data, test_filenames, test_labels, label_names = load_cifar_10_data('cifar-10-batches-py')
# Download data


In [16]:
predictions_snn = snn_converter.predict(test_data[:100], duration_per_sample=100)

100%|██████████| 100/100 [01:51<00:00,  1.11s/it]


In [19]:
from sklearn.metrics import classification_report, accuracy_score
import numpy as np

# Convert one-hot labels (returned by load_cifar_10_data) to integer class indices
y_true = test_labels[:100]
if y_true.ndim == 2 and y_true.shape[1] > 1:
    y_true = np.argmax(y_true, axis=1)

print(classification_report(y_true, predictions_snn))
print("Test accuracy of the SNN:", accuracy_score(y_true, predictions_snn))

              precision    recall  f1-score   support

           0       0.13      0.20      0.16        10
           1       0.17      0.17      0.17         6
           2       0.00      0.00      0.00         8
           3       0.20      0.20      0.20        10
           4       0.00      0.00      0.00         7
           5       0.13      0.25      0.17         8
           6       0.22      0.12      0.16        16
           7       0.00      0.00      0.00        11
           8       0.18      0.15      0.17        13
           9       0.17      0.09      0.12        11

    accuracy                           0.12       100
   macro avg       0.12      0.12      0.11       100
weighted avg       0.13      0.12      0.12       100

Test accuracy of the SNN: 0.12


In [15]:
# Recompile the model to fix any potential metrics compilation issues
model.compile(
    optimizer='sgd',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
test_metrics = model.evaluate(x=test_data, y=test_labels, batch_size=50, verbose=1, return_dict=True)
print("Accuracy keras: ", test_metrics['accuracy'])

200/200 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.8719 - loss: 0.4542
Accuracy keras:  0.8719000220298767
